In [2]:
import PyPDF2

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp


from langchain.embeddings import HuggingFaceEmbeddings # import hf embedding
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer, util
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Step 1: Preparing pdf metadata

In [10]:
pdf_files=["C:/Users/Mrinal Kalita/Python Projects/AIML Capstone Project - CV - Pneumonia Detection-1.pdf"]

In [13]:
def process_pdf(pdf_files):
    documents = []
    metadata = []
    content = []

    for i in pdf_files:

        pdf_read = PyPDF2.PdfReader(i)
        for ind, text in enumerate(pdf_read.pages):
            doc_page = {'title': i + " page " + str(ind + 1),
                        'content': pdf_read.pages[ind].extract_text()}
            documents.append(doc_page)
    for doc in documents:
        content.append(doc["content"])
        metadata.append({
            "title": doc["title"]
        })
    print("Content and metadata are extracted from the documents")
    return content, metadata

In [17]:
content, metadata = process_pdf(pdf_files)

Content and metadata are extracted from the documents


# Step 2: Split the content into smaller portion

In [15]:
def split_content(content, metadata):
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512,
        chunk_overlap=256,
    )
    smaller_docs = splitter.create_documents(content, metadatas=metadata)
    print(f"Documents are split into {len(smaller_docs)} passages")
    return smaller_docs

In [18]:
split_content(content, metadata)

Documents are split into 7 passages


[Document(page_content='COMPUTER VISION\n©Great Learning. Proprietary content. All Rights Reserved. Unauthorised use or distribution prohibited\nCAPSTONE PROJECTAIML \nPNEUMONIA DETECTION CHALLENGE\nkalita278@gmail.com\nFSL1MRBPOT\nThis file is meant for personal use by kalita278@gmail.com only.\nSharing or publishing the contents in part or full is liable for legal action.', metadata={'title': 'C:/Users/Mrinal Kalita/Python Projects/AIML Capstone Project - CV - Pneumonia Detection-1.pdf page 1'}),
 Document(page_content='CAPSTONE PROJECT - CV', metadata={'title': 'C:/Users/Mrinal Kalita/Python Projects/AIML Capstone Project - CV - Pneumonia Detection-1.pdf page 2'}),
 Document(page_content='©Great Learning. Proprietary content. All Rights Reserved. Unauthorised use or distribution prohibitedPage 1PROBLEM STA TEMENT •DOMAIN: Health Care  •CONTEXT:  Computer vision can be used in health care for identifying diseases. In Pneumonia detection we need to detect Inflammation of the lungs. In